This notebook made for testing purpose. It contains NDVI and EVI maps using folium as well as geemap, land cover over Unites States using NLCD 2016 CONUS Land Cover basemap, land cover over the world using MODIS/051/MCD12Q1/2013_01_01 dataset with legends, timelapse for NDVI and widgets





In [1]:
!pip install geehydro

  Created wheel for geehydro: filename=geehydro-0.2.0-py2.py3-none-any.whl size=10141 sha256=9d8584212183bef90da2eeb00f9ed6398d8bdf37eb76d0cf01b460f157d8c6c2
  Stored in directory: /root/.cache/pip/wheels/49/6c/92/019aeb37663078e6666afa9dd336e2d8df72d5683d7d357697
Successfully built geehydro


In [2]:
!pip install geemap

     |████████████████████████████████| 493 kB 31.6 MB/s 
     |████████████████████████████████| 8.6 MB 39.9 MB/s 
     |████████████████████████████████| 219 kB 59.1 MB/s 
     |████████████████████████████████| 93 kB 1.4 MB/s 
     |████████████████████████████████| 93 kB 1.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 130 kB 59.9 MB/s 
     |████████████████████████████████| 1.3 MB 62.0 MB/s 
     |████████████████████████████████| 76 kB 5.3 MB/s 
     |████████████████████████████████| 1.6 MB 47.3 MB/s 
     |████████████████████████████████| 3.3 MB 38.8 MB/s 
     |████████████████████████████████| 98 kB 7.0 MB/s 
     |████████████████████████████████| 1.2 MB 65.1 MB/s 
     |████████████████████████████████| 216 kB 54.0 MB/s 
     |████████████████████████████████| 97 kB 6.4 MB/s 
     |████████████████████████████████| 428 kB 70.0 MB/s 
     |████████

In [3]:
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [5]:
# l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
point = ee.Geometry.Point([-122.292, 37.9018])
# image = ee.Image(
#   l8.filterBounds(point)
#     .filterDate('2021-05-01', '2021-05-31')
#     .sort('CLOUD_COVER')
#     .first()
# )
# print(image)

image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2021-10-01', '2021-10-31').min()
print(image)

ee.Image({
  "functionInvocationValue": {
    "functionName": "reduce.min",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.filter",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "ImageCollection.load",
                "arguments": {
                  "id": {
                    "constantValue": "LANDSAT/LC08/C01/T1_SR"
                  }
                }
              }
            },
            "filter": {
              "functionInvocationValue": {
                "functionName": "Filter.dateRangeContains",
                "arguments": {
                  "leftValue": {
                    "functionInvocationValue": {
                      "functionName": "DateRange",
                      "arguments": {
                        "end": {
                          "constantValue": "2021-10-31"
                        },
               

In [6]:
import folium
import geehydro
Map = folium.Map(location=[20, 0], zoom_start=3)
Map.setOptions('HYBRID')

In [7]:
nir = image.select('B5')
red = image.select('B4')

ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')

ndviParams = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}

In [8]:
Map.addLayer(ndvi, ndviParams, 'NDVI image')
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)

In [9]:
#NDVI map with folium
Map

In [10]:
Map1 = folium.Map(location=[20, 0], zoom_start=3)

Map1.setOptions('HYBRID')

In [11]:
evi = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR': image.select('B5'),
      'RED': image.select('B4'),
      'BLUE': image.select('B2')
})
eviParams = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
# EVI map using with folium
Map1.addLayer(evi, eviParams, 'EVI image')
Map1.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map1

In [12]:
import geemap

m = geemap.Map(center=[40,-100], zoom=4)
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [13]:
m.add_basemap("NLCD 2016 CONUS Land Cover")

In [14]:
map = geemap.Map()

ee_class_table = """

Value	Color	Description
0	1c0dff	Water
1	05450a	Evergreen needleleaf forest
2	086a10	Evergreen broadleaf forest
3	54a708	Deciduous needleleaf forest
4	78d203	Deciduous broadleaf forest
5	009900	Mixed forest
6	c6b044	Closed shrublands
7	dcd159	Open shrublands
8	dade48	Woody savannas
9	fbff13	Savannas
10	b6ff05	Grasslands
11	27ff87	Permanent wetlands
12	c24f44	Croplands
13	a5a5a5	Urban and built-up
14	ff6d4c	Cropland/natural vegetation mosaic
15	69fff8	Snow and ice
16	f9ffa4	Barren or sparsely vegetated
254	ffffff	Unclassified

"""

landcover = ee.Image('MODIS/051/MCD12Q1/2013_01_01') \
    .select('Land_Cover_Type_1')
map.setCenter(6.746, 46.529, 2)
map.addLayer(landcover, {}, 'MODIS Land Cover')

legend_dict = geemap.legend_from_ee(ee_class_table)
map.add_legend(legend_title="MODIS Global Land Cover", legend_dict=legend_dict)

map

Map(center=[46.529, 6.746], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

In [15]:
#NDVI and EVI with geemap
map1 = geemap.Map(center=[40,-100], zoom=4)
map1.addLayer(ndvi, ndviParams, 'NDVI image')
map1.addLayer(evi, eviParams, 'EVI image')
# legend_keys = ['blue', 'white', 'dark green', 'light green']
# legend_colors = ['#0000FF','#FFFFFF','#008000','#7CFC00']
# map1.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomleft')
map1

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [20]:
import ipyleaflet
import ipywidgets as widgets

int_slider = widgets.IntSlider(
    value=2014,
    min=2014,
    max=2020,
    step=1,
    description='Start Year:'
)
int_slider

int_slider1 = widgets.IntSlider(
    value=2015,
    min=2014,
    max=2020,
    step=1,
    description='End Year:'
)
int_slider1




IntSlider(value=2015, description='End Year:', max=2020, min=2014)

In [21]:
label="Timelapse for NDVI"
map1.add_landsat_ts_gif(label= label, bands=['Blue', 'NIR', 'Red'], nd_bands=['NIR', 'Red'], nd_palette=['blue', 'white', 'green'], nd_threshold=0.3, start_year=int_slider.value, end_year=int_slider1.value, frames_per_second=5)

Generating URL...
Please wait ...
An error occurred while downloading.
Generating URL...
Please wait ...
The GIF image has been saved to: /root/Downloads/landsat_ts_ilx_nd.gif
Adding animated text to GIF ...
The input gif file does not exist.
The input gif file does not exist.
The input gif file does not exist.
Adding GIF to the map ...
The provided file does not exist.
The timelapse has been added to the map.


In [18]:
import ipywidgets as widgets
style = {'description_width': 'initial'}

dropdown = widgets.Dropdown(
    options=['USA', 'Canada', 'Mexico'],
    value='Canada',
    description='Country:'
)

hbox2 = widgets.HBox([dropdown])
hbox2

In [19]:
dropdown.value

'Canada'